In [1]:
import os
import tweepy as tw
import pandas as pd

Using this site as reference: https://www.earthdatascience.org/courses/use-data-open-source-python/intro-to-apis/twitter-data-in-python/

# Step 1: Set up Twitter API access
Set up the project here: https://developer.twitter.com/en/portal/projects-and-apps

In [2]:
# Define access keys and tokens
# TODO: mask credentials
consumer_key= ''
consumer_secret= ''
access_token= ''
access_token_secret= ''

In [3]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit = True)

# Step 2: Get tweets and transform into dataframe

## Method 1: Tweets to Dataframe

In [4]:
search_word = "#covidcanada"
num_of_tweets = 120
include_retweets = False

if include_retweets == False:
    search_word = search_word + " -filter:retweets"

tweets = api.search(q=search_word, count=num_of_tweets)

json_data = [t._json for t in tweets]

# create dataframe from JSON result
df = pd.json_normalize(json_data)
df.head()

,created_at,id,id_str,text,truncated,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,quoted_status.favorited,quoted_status.retweeted,quoted_status.possibly_sensitive,quoted_status.lang,entities.media,extended_entities.media,quoted_status.entities.media,quoted_status.extended_entities.media,quoted_status.quoted_status_id,quoted_status.quoted_status_id_str
0,Thu May 27 18:23:37 +0000 2021,1397981837096411136,1397981837096411136,"At current rates, when will most Canadians be ...",True,"<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Thu May 27 18:20:04 +0000 2021,1397980943294603266,1397980943294603266,University in Ontario becomes first to require...,True,"<a href=""https://www.hootsuite.com"" rel=""nofol...",NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Thu May 27 18:20:04 +0000 2021,1397980942606684164,1397980942606684164,University in Ontario becomes first to require...,True,"<a href=""https://www.hootsuite.com"" rel=""nofol...",NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Thu May 27 18:20:03 +0000 2021,1397980938177593346,1397980938177593346,University in Ontario becomes first to require...,True,"<a href=""https://www.hootsuite.com"" rel=""nofol...",NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Thu May 27 17:53:46 +0000 2021,1397974324699701248,1397974324699701248,Dylan's @nofrillsCA in Spryfield IS NOT ENFORC...,True,"<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# print all column names
for col in df.columns:
    print(col)

created_at
id
id_str
text
truncated
source
in_reply_to_status_id
in_reply_to_status_id_str
in_reply_to_user_id
in_reply_to_user_id_str
in_reply_to_screen_name
geo
coordinates
place
contributors
is_quote_status
retweet_count
favorite_count
favorited
retweeted
possibly_sensitive
lang
entities.hashtags
entities.symbols
entities.user_mentions
entities.urls
metadata.iso_language_code
metadata.result_type
user.id
user.id_str
user.name
user.screen_name
user.location
user.description
user.url
user.entities.url.urls
user.entities.description.urls
user.protected
user.followers_count
user.friends_count
user.listed_count
user.created_at
user.favourites_count
user.utc_offset
user.time_zone
user.geo_enabled
user.verified
user.statuses_count
user.lang
user.contributors_enabled
user.is_translator
user.is_translation_enabled
user.profile_background_color
user.profile_background_image_url
user.profile_background_image_url_https
user.profile_background_tile
user.profile_image_url
user.profile_image_url_h

In [60]:
df.lang.unique

<bound method Series.unique of 0     en
1     en
2     en
3     en
4     en
      ..
95    en
96    en
97    en
98    fr
99    en
Name: lang, Length: 100, dtype: object>

In [61]:
df_english = df.loc[df['lang'] == 'en']

min_date = df_english.created_at.min()
max_date = df_english.created_at.max()

print('Min. Date: ', min_date, '\nMax. Date: ', max_date)

Min. Date:  Tue May 18 11:35:58 +0000 2021 
Max. Date:  Wed May 19 05:00:04 +0000 2021


In [62]:
df_english.count()

created_at                               93
id                                       93
id_str                                   93
text                                     93
truncated                                93
                                         ..
place.bounding_box.type                   1
place.bounding_box.coordinates            1
quoted_status.user.entities.url.urls      4
quoted_status.entities.media              2
quoted_status.extended_entities.media     2
Length: 310, dtype: int64

## Method 2: Tweets to Dataframe

In [29]:
# https://www.earthdatascience.org/courses/use-data-open-source-python/intro-to-apis/twitter-data-in-python/
search_word = "#covidcanada"
num_of_tweets = 1000
include_retweets = False
language = "en"

if include_retweets == False:
    search_word = search_word + " -filter:retweets"

tweets = tw.Cursor(api.search,
                    q=search_word,
                    lang=language).items(num_of_tweets)

In [30]:
tweet_metadata = [[tweet.created_at, tweet.id, tweet.text, tweet.user.screen_name, tweet.retweet_count, tweet.favorite_count] for tweet in tweets]

print(tweet_metadata[:2])

[[datetime.datetime(2021, 5, 19, 5, 0, 4), 1394880514641780736, 'It’s no longer a question of if we should have vaccine passports in Canada but how they should be designed… https://t.co/subT7kmPpP', 'georgiastraight', 0, 4], [datetime.datetime(2021, 5, 19, 3, 0, 8), 1394850329158557699, "Canada's COVID-19 vaccination rate likely to surpass U.S. this week https://t.co/ZkBBjt8LKJ via @CTVNews… https://t.co/9CgGdqayLF", 'iChinadian', 0, 0]]


In [26]:
df_tweets = pd.DataFrame(data=tweet_metadata, columns=['created_at', 'id', 'text', 'user', 'rt_count', 'fav_count'])
df_tweets.head()

,created_at,id,text,user,rt_count,fav_count
0,2021-05-19 05:00:04,1394880514641780736,It’s no longer a question of if we should have...,georgiastraight,0,4
1,2021-05-19 03:00:08,1394850329158557699,Canada's COVID-19 vaccination rate likely to s...,iChinadian,0,0
2,2021-05-19 02:03:43,1394836134060986372,MAY 18\n\nRACE ENDS MAY 31: ANNIVERSARY of CAN...,Fifthmeridian,0,0
3,2021-05-19 02:03:43,1394836132265791489,"MAY 18\n\nWEST wins 14(W)-9(E)-1(N), 6/12 podi...",Fifthmeridian,0,0
4,2021-05-19 01:59:24,1394835045773242368,COVID-19: Fraser Health accounts for 75 per ce...,iChinadian,0,0


In [36]:
print('Rows:', df_tweets.shape[0], '\nColumns:', df_tweets.shape[1])

Rows: 299 
Columns: 6


In [38]:
min_date = df_tweets.created_at.min()
max_date = df_tweets.created_at.max()

print('Min. date: ', min_date, '\nMax. date: ', max_date)

Min. date:  2021-05-11 16:44:45 
Max. date:  2021-05-19 05:00:04


# Step 3: Analyze tweets

In [1]:
from nltk.corpus import stopwords

In [53]:
# sample tweets
for i in range(0,5):
    print(str(i+1) + ') ' + df_tweets.text[i] + '\n')

1) It’s no longer a question of if we should have vaccine passports in Canada but how they should be designed… https://t.co/subT7kmPpP

2) Canada's COVID-19 vaccination rate likely to surpass U.S. this week https://t.co/ZkBBjt8LKJ via @CTVNews… https://t.co/9CgGdqayLF

3) MAY 18

RACE ENDS MAY 31: ANNIVERSARY of CANADA's DEADLIEST DAY of the Covid pandemic

TOTAL INFECTIONS
(% of pop i… https://t.co/lK62KDXl4r

4) MAY 18

WEST wins 14(W)-9(E)-1(N), 6/12 podiums

AB: 1st-DAILY CASES (32 of 35 days)
SK: 1st-DAILY DEATHS
NB: 2nd-D… https://t.co/10sCv9f7Kd

5) COVID-19: Fraser Health accounts for 75 per cent of new cases reported Tuesday https://t.co/9cIJ1ywitI #COVIDBC… https://t.co/bbdGBcTppq



## Step 3a: Tokenization

## Step 3b: Remove stop words